In [1]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re
import ast 
import json

In [2]:
list_patents = pd.read_csv('./Data/patents_number.csv',index_col=None)

In [3]:
L = list(list_patents.patent_number.values)

In [4]:
df = pd.DataFrame(columns=['patent_number', 'patent_title', 'patent_date', 'patent_kind',
       'patent_type', 'assignees'])

In [5]:
L[-1]

99854890

In [6]:
i=0
while i < (len(L)-700):    
    token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
    token2 = '}&f=["patent_number","patent_title","patent_date","patent_kind","patent_type","assignee_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L[i:i+700]) + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["patents"]
    df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
    i+=700
    print(i)
token1 ='https://api.patentsview.org/patents/query?q={"patent_number":'
token2 = '}&f=["patent_number","patent_title","patent_date","patent_kind","patent_type","assignee_id"]&s=[{"patent_title":"asc"}]&o={"page":1,"per_page":1000}'
url = token1 + str(L[i:]) + token2
r = requests.get(url)
html=r.text
html = html.replace("null","None")
Dict = ast.literal_eval(html)["patents"]
df = pd.concat([df,pd.read_json(json.dumps(Dict),orient='records')])
df = df.reset_index(drop=True)

700
1400
2100
2800
3500
4200
4900
5600
6300
7000
7700
8400
9100
9800
10500
11200
11900
12600
13300
14000
14700
15400
16100
16800
17500
18200
18900
19600
20300
21000
21700
22400
23100
23800


In [7]:
df

,patent_number,patent_title,patent_date,patent_kind,patent_type,assignees
0,5387603,"1,5,7-trisubstituted indoline compounds and sa...",1995-02-07,A,utility,[{'assignee_id': '5330279e-a21e-438c-80d1-7e6a...
1,5771295,5-2-5 matrix system,1998-06-23,A,utility,[{'assignee_id': '2986e46e-7937-477f-9bc5-1f5e...
2,5585587,Acoustic image localization apparatus for dist...,1996-12-17,A,utility,[{'assignee_id': 'f7152515-d420-42d9-a7b6-3a27...
3,5321490,Active near-field object sensor and method emp...,1994-06-14,A,utility,[{'assignee_id': 'ec72750e-f789-4749-91f7-6292...
4,5750497,Acylated insulin,1998-05-12,A,utility,[{'assignee_id': '9a3afa40-66e5-4921-b831-6428...
...,...,...,...,...,...,...
23905,10299691,Wearable monitor with arrhythmia burden evalua...,2019-05-28,B2,utility,[{'assignee_id': '1b2d67d2-f1f9-4c29-8cd1-a5e7...
23906,10292890,Wearable thorax percussion device,2019-05-21,B2,utility,[{'assignee_id': 'f0997ba8-1ec2-4c0c-aaea-d800...
23907,10299968,Wireless incontinence detection apparatus,2019-05-28,B2,utility,[{'assignee_id': 'e5fc32c3-a913-455a-94ce-9932...
23908,10149616,Wireless patient monitoring device,2018-12-11,B2,utility,[{'assignee_id': '4eb39673-7f59-4c45-bf6c-2275...


In [8]:
assignee_id=[]

for index,j in enumerate(df.assignees.values):
    assigne = j[0]
    assignee_id.append(assigne["assignee_id"]) 
    #assignee_type.append(assigne['assignee_type'])
df["assignee_id"] = assignee_id
#df["assignee_type"] =assignee_type


In [9]:
df = df.drop(['assignees','patent_type'],axis=1)

In [10]:
df

,patent_number,patent_title,patent_date,patent_kind,assignee_id
0,5387603,"1,5,7-trisubstituted indoline compounds and sa...",1995-02-07,A,5330279e-a21e-438c-80d1-7e6ae793c092
1,5771295,5-2-5 matrix system,1998-06-23,A,2986e46e-7937-477f-9bc5-1f5eec05183f
2,5585587,Acoustic image localization apparatus for dist...,1996-12-17,A,f7152515-d420-42d9-a7b6-3a2796bc5d10
3,5321490,Active near-field object sensor and method emp...,1994-06-14,A,ec72750e-f789-4749-91f7-6292f7c6fda7
4,5750497,Acylated insulin,1998-05-12,A,9a3afa40-66e5-4921-b831-64287f2052e0
...,...,...,...,...,...
23905,10299691,Wearable monitor with arrhythmia burden evalua...,2019-05-28,B2,1b2d67d2-f1f9-4c29-8cd1-a5e75ba27be7
23906,10292890,Wearable thorax percussion device,2019-05-21,B2,f0997ba8-1ec2-4c0c-aaea-d80066a49317
23907,10299968,Wireless incontinence detection apparatus,2019-05-28,B2,e5fc32c3-a913-455a-94ce-9932d30eefff
23908,10149616,Wireless patient monitoring device,2018-12-11,B2,4eb39673-7f59-4c45-bf6c-2275dccf9048


In [11]:
df_assignees=pd.DataFrame(columns=["assignee_id", "assignee_organization","assignee_type", "assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"])

In [12]:
L_tmp = list(np.unique(df.assignee_id.dropna().values))
L2 = []
k=0
while k < len(L_tmp)-100:
    L2.append(json.dumps(L_tmp[k:k+100]))
    k+=100
L2.append(json.dumps(L_tmp[k:]))

In [13]:
for i in range(len(L2)):    
    token1 ='https://api.patentsview.org/assignees/query?q={"assignee_id":'
    token2 = '}&f=["assignee_id","assignee_organization","assignee_type","assignee_lastknown_country", "assignee_total_num_patents", "assignee_total_num_inventors"]&s=[{"assignee_organization":"asc"}]&o={"page":1,"per_page":1000}'
    url = token1 + str(L2[0]) + token2
    r = requests.get(url)
    html=r.text
    html = html.replace("null","None")
    Dict = ast.literal_eval(html)["assignees"]
    df_assignees = pd.concat([df_assignees,pd.read_json(json.dumps(Dict),orient='records')])
    print(i*100)
df_assignees=df_assignees.reset_index(drop=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [14]:
df_assignees

,assignee_id,assignee_organization,assignee_type,assignee_lastknown_country,assignee_total_num_patents,assignee_total_num_inventors
0,02caa49b-3ff8-4984-8d34-b8a19691104d,None,4,US,3,12
1,00e0a2a8-b543-4cb3-b25c-df250a972be4,2236008 Ontario Inc.,3,CA,162,170
2,0a5c0b7a-1f74-4544-b210-66751ca7f8a6,"AAT Catheter Technologies, LLC",2,US,1,2
3,0136a7f6-1083-409a-a521-da27a9d652cd,Activiews Ltd.,3,IL,2,1
4,0a6e0410-2083-4dc8-ab35-c176e4fe79c5,Actuant Corporation,2,US,198,174
...,...,...,...,...,...,...
2295,06264f32-6874-4d24-9d34-97260dce09f0,VIOLIN MEMORY LLC,2,US,59,27
2296,03c29547-cdab-4b92-9074-d75207d6bd63,"Wall Ace, LLC",2,US,21,22
2297,04048aa6-ff9e-4a59-93a8-fcf535f29b59,"Watson Laboratories, Inc.",2,US,89,73
2298,0261cac5-7dde-4e53-9218-05303747126a,"Xytrans, Inc.",2,US,37,13


In [15]:
df.to_csv('./info_patents.csv')
df_assignees.to_csv('./info_companies.csv')